### Packages and variables

In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db2 = "fcsq"
#pub_year = 2011
#pub_qtr = 4

### CSV creation stages

#### Applications/Starts

In [ ]:
#Applications by type - children involved
print("aggregating child level dataset....")
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Children - orders applied for' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  Order_desc AS Order_type,
  Order_code AS Order_type_code,  
  app_dfj AS dfj,
  app_region AS region,  
  COUNT(*) AS Count
FROM
  {db2}.ca_apps_child
WHERE 
  year > 2010
GROUP BY
  Year,
  Quarter,
  order_case_type,
  Order_desc,
  Order_code,
  app_dfj,
  app_region
""",

"pbi_ca_apps_type_child")

In [ ]:
#Orders applied for (by order type)
print("aggregating orders applied for dataset....")
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Orders applied for' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  Order_desc AS Order_type,
  Order_code AS Order_type_code,  
  app_dfj AS dfj,
  app_region AS region,  
  COUNT(*) AS Count
FROM
  {db2}.ca_apps_order_types
WHERE 
  year > 2010  
GROUP BY
  Year,
  Quarter,
  order_case_type,
  Order_desc,
  Order_code,
  app_dfj,
  app_region
""",

"pbi_ca_apps_type_order")

In [ ]:
#Application events
print("aggregating applications dataset....")
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Applications' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
   app_dfj AS dfj,
   app_region AS region,
   COUNT(*) AS Count
FROM
  {db2}.ca_applications
WHERE 
  year > 2010  
GROUP BY
  Year,
  Quarter,
  order_case_type,
  app_dfj,
  app_region
""",

"pbi_ca_apps_events")

In [ ]:
#Case starts - including number of applicants/respondents and Hight court indicator*/
print("aggregating case starts dataset....")
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Cases starting' AS Count_type,
  CASE WHEN main_case_type = 'C' THEN 'Public law'
       WHEN main_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
   dfj,
   region,
   COUNT(*) AS Count
FROM
  {db2}.ca_case_starts
WHERE 
  year > 2010  
GROUP BY
  Year,
  Quarter,
  main_case_type,
  dfj,
  region
""",

"pbi_ca_apps_cases")

#### Disposals/Cases closed

In [ ]:
#Children involved in orders made
print("aggregating child level dataset....")
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Children - orders made' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   order_desc AS Order_type,
   order_code AS Order_type_code,
   disposal_dfj AS dfj,
   disposal_region AS region,
   COUNT(*) AS Count
FROM
  {db2}.ca_disps_all_children
WHERE
  year > 2010
  AND disp_type_code = 1
GROUP BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  order_desc,
  order_code,
  disposal_dfj,
  disposal_region
  
""",

"pbi_order_child_counts")

In [ ]:
#Orders made
print("aggregating order/disposal level dataset....")
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Orders made' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   order_desc AS Order_type,
   order_code AS Order_type_code,
   disposal_dfj AS dfj,
   disposal_region AS region,
   COUNT(*) AS Count
FROM
  {db2}.ca_all_disposals
WHERE
  year > 2010  
  AND disp_type_code = 1
GROUP BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  order_desc,
  order_code,
  disposal_dfj,
  disposal_region
  
""",

"pbi_order_type_counts")

In [ ]:
#Closed cases
print("aggregating case level dataset....")
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) AS Quarter,
  'Cases closed' AS Count_type,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  case_closed_dfj AS dfj,
  case_closed_region AS region,
  COUNT(*) AS Count
FROM
  {db2}.ca_closed_cases
WHERE
  year > 2010  
GROUP BY
  Year,
  Quarter,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    END,
  case_closed_dfj,
  case_closed_region
  
""",

"pbi_disp_case_counts")

In [ ]:
print("creating children act disposals csv....")
pydb.create_temp_table( 
f"""
SELECT
  *
FROM
  __temp__.PBI_CA_APPS_TYPE_CHILD
UNION ALL
SELECT
  *
FROM
  __temp__.PBI_CA_APPS_TYPE_ORDER
UNION ALL
SELECT
  *
FROM
  __temp__.PBI_CA_APPS_EVENTS
UNION ALL
SELECT
  *
FROM
  __temp__.PBI_CA_APPS_CASES
UNION ALL
SELECT
  *
FROM
  __temp__.pbi_order_child_counts
UNION ALL
SELECT
  *
FROM
  __temp__.pbi_order_type_counts
UNION ALL
SELECT
  *
FROM
  __temp__.pbi_disp_case_counts

""",

"pbi_ca_csv")

### Exporting data

In [ ]:
pbi_ca_csv_data = pydb.read_sql_query("SELECT * FROM __temp__.pbi_ca_csv WHERE Public_private = 'Private law'")

In [ ]:
pbi_ca_csv_df = pd.DataFrame(pbi_ca_csv_data)

In [ ]:
pbi_ca_csv_df.to_csv (r's3://alpha-family-data/power_bi/pbi_ca_csv.csv', header = True, index=False)